In [3]:
import numpy as np
import pandas as pd

df = pd.read_excel('airfoils.xlsx')

#show what we just loaded
df

,NACA64_A17 x,NACA64_A17 y
0,1.000,0.000000
1,0.990,0.003385
2,0.980,0.006126
3,0.970,0.008767
4,0.960,0.011357
5,0.950,0.013921
6,0.940,0.016478
7,0.930,0.019036
8,0.920,0.021598
9,0.910,0.024163


In [4]:
from scipy import interpolate

af_namex='NACA64_A17 x'
af_namey='NACA64_A17 y'


x_vals=np.array(df[af_namex])
y_vals=np.array(df[af_namey])

#find the first line with nan or stop if we reach the end of the np array
stopc=0
cnt=0
while stopc==0:
    if cnt==np.size(x_vals):
        stopc=1
        cnt=cnt-1
        break
    if (np.isnan(x_vals[cnt])):
        stopc=1
        cnt=cnt-2
    cnt=cnt+1
dwn_cnt=cnt
dwn_cnt_init=dwn_cnt
up_cnt=0

#find the zero index
stopc=0
cnt=0
while stopc==0:
    if abs(x_vals[cnt])<=1E-6:
        stopc=1
        cnt=cnt-1
    cnt=cnt+1
zero_ind=cnt

max_thick=0.0
max_thick_x=0.0

#create the interpolation classes
interp_up=interpolate.interp1d(x_vals[0:zero_ind],y_vals[0:zero_ind],fill_value="extrapolate")
interp_dwn=interpolate.interp1d(x_vals[zero_ind:dwn_cnt_init],y_vals[zero_ind:dwn_cnt_init],fill_value="extrapolate")

while (abs(x_vals[dwn_cnt])>1E-6 or abs(x_vals[up_cnt])>1E-6):
    #we use the the x-value of pressure or suction side, depending on which one has a finer disrectiation
    max_x_pos=max(abs(x_vals[dwn_cnt]),abs(x_vals[up_cnt]))
    
    #interpolate:
    y_val_up_cnt=interp_up(max_x_pos)
    y_val_dwn_cnt=interp_dwn(max_x_pos)
    
    #calculate the thickness
    thick=max(y_val_up_cnt,y_val_dwn_cnt)-min(y_val_up_cnt,y_val_dwn_cnt)
   
    #check if this is the max thickness for this airfoil
    if thick>max_thick:
        max_thick=thick
        max_thick_x=max_x_pos
         
    #check if we are close to x=0 and move to next point in case we are not        
    if abs(x_vals[dwn_cnt])>1E-6:
        dwn_cnt=dwn_cnt-1
    
    #check if we are close to x=0 and move to next point in case we are not 
    if abs(x_vals[up_cnt])>1E-6:
        up_cnt=up_cnt+1
    
print("max_x_pos:",max_thick_x,"max_thickness:",max_thick)

max_x_pos: 0.37 max_thickness: 0.179986
